What's a trace, a choicemap, and how do I select from it? 

Accessing the right elements in the trace can become non-trivial when one creates hierarchical generative functions. 
Here's a minimal example and solution for selection.

In [10]:

# TODO: more basic examples, choicemap creation, and high level explanations
from genjax import cond_combinator, bernoulli, gen, vmap_combinator, normal, scan_combinator, repeat_combinator, mixture_combinator
import jax.numpy as jnp
import jax.random as random
from jax import jit
from genjax import ChoiceMapBuilder as C

key = random.PRNGKey(0)

### Choice map creation

# Create a choice map with a single choice 0.5 at the address "p"
chm = C["p"].set(0.5)
# Create a choice map at several addresses
chm = C["p"].set(0.5) ^ C["v"].set(1)
# Alternatively
chm2 = C["p"].set(0.5).at["v"].set(1)
# One can iteratively add choices to a choice map
chm2 = chm2 ^ C["p2"].set(0.6)
for i in range(10):
   chm2 = chm2 ^ C["p"+str(i)].set(i)

### Trace selection in hierarchical models

# For `cond_combinator`
@gen
def model(p):
    branch_1 = gen(lambda p: bernoulli(p) @ "v1")
    branch_2 = gen(lambda p: bernoulli(-p) @ "v2")
    v = cond_combinator(branch_1, branch_2)(jnp.int32(p > 0), (p,), (p,)) @ "s"
    return v

trace = jit(model.simulate)(key, (0.5,))
print(trace.get_sample()[("s", "v2")])
print()

# For `vmap_combinator`
sample_image = vmap_combinator(in_axes=(0,))(vmap_combinator(in_axes=(0,))(gen(lambda pixel: normal(pixel,1.0) @ "new_pixel")))

image = jnp.zeros([2,3], dtype=jnp.float32)
trace = sample_image.simulate(key,(image,))
print(trace.get_sample()[...,...,"new_pixel"])
print()

# For `scan_combinator`
@scan_combinator(max_length=10)
@gen
def hmm(x, c):
    z = normal(x, 1.0) @ "z"
    y = normal(z, 1.0) @ "y"
    return y, None
trace = hmm.simulate(key, (0.0, None))
print(trace.get_sample()[..., "z"])
print(trace.get_sample()[3,"y"])
print()

# For `repeat_combinator`
@repeat_combinator(num_repeats=10)
@gen
def model(y):
    x = normal(y, 0.01) @ "x"
    y = normal(x, 0.01) @ "y"
    return y
trace = model.simulate(key, (0.3,))
print(trace.get_sample()[...,"x"])
print()

# For `mixture_combinator`
@gen
def mixture_model(p):
    z = normal(p, 1.0) @ "z"
    # the switch combinators take as input the logits of the mixture components
    logits = (0.3, 0.5, 0.2)
    # and args for each component of the mixture
    arg_1 = (p,)
    arg_2 = (p,)
    arg_3 = (p,)
    a = mixture_combinator(
        gen(lambda p: normal(p, 1.0) @ "x1"), 
        gen(lambda p: normal(p, 2.0) @ "x2"), 
        gen(lambda p: normal(p, 3.0) @ "x3"))(logits, arg_1, arg_2, arg_3) @ "a"
    return a+z

trace = mixture_model.simulate(key, (0.4,))
# The combinator uses a fixed address "mixture_component" for the components of the mixture model. 
print(trace.get_sample()["a","mixture_component"])

Mask(flag=<jax.Array(True, dtype=bool)>, value=<jax.Array(0, dtype=int32)>)

[[ 1.1605581   0.95527214 -1.7698178 ]
 [-0.37148237  1.1890742  -0.6553323 ]]

[ 0.36448452  0.25174013 -0.7845038  -0.85613173 -1.9163786  -0.56642795
 -0.96378577  0.6517954  -0.17539416  0.32161352]
Mask(flag=<jax.Array(True, dtype=bool)>, value=<jax.Array(-2.0501354, dtype=float32)>)

[0.30189174 0.31286433 0.3139801  0.32622513 0.31059152 0.3164824
 0.30230102 0.30271375 0.30742478 0.26852012]



TypeError: ExactDensityFromCallables.sample() got an unexpected keyword argument 'logits'